<a href="https://colab.research.google.com/github/yesung05/25-BDA-TeamDP/blob/main/25_BDA_BikeCrash_TeamDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install koreanize_matplotlib

In [ ]:
import koreanize_matplotlib

In [ ]:
rData = pd.read_csv('bike_data.csv', encoding = 'cp949') #rawData
rData.head(3)

,다발지구분,다발지시군구,지점명,발생일,발생시간대,사고내용,가해자성별,가해자연령,가해자차종,가해자신체상해정도,피해자성별,피해자연령,피해자차종,피해자신체상해정도,사고유형,법규위반사항,기상상태,사망자수,부상자수
0,2018년 다발지,서울특별시 종로구1,서울특별시 종로구 종로6가(흥인지문사거리 부근),2018-03-23,07시,경상,남,59세,자전거,상해없음,남,40세,보행자,경상,차대사람 - 횡단중,신호위반,맑음,0,5
1,2018년 다발지,서울특별시 종로구1,서울특별시 종로구 종로6가(흥인지문사거리 부근),2018-03-25,17시,부상신고,남,57세,자전거,부상신고,남,75세,승용차,상해없음,차대차 - 측면충돌,신호위반,맑음,0,5
2,2018년 다발지,서울특별시 종로구1,서울특별시 종로구 종로6가(흥인지문사거리 부근),2018-06-23,14시,중상,남,78세,자전거,중상,남,63세,승용차,상해없음,차대차 - 측면충돌,신호위반,맑음,0,5


In [ ]:
rData.shape

(4916, 19)

In [ ]:
rData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   다발지구분      4916 non-null   object
 1   다발지시군구     4916 non-null   object
 2   지점명        4916 non-null   object
 3   발생일        4916 non-null   object
 4   발생시간대      4916 non-null   object
 5   사고내용       4916 non-null   object
 6   가해자성별      4916 non-null   object
 7   가해자연령      4916 non-null   object
 8   가해자차종      4916 non-null   object
 9   가해자신체상해정도  4916 non-null   object
 10  피해자성별      4916 non-null   object
 11  피해자연령      4916 non-null   object
 12  피해자차종      4916 non-null   object
 13  피해자신체상해정도  4916 non-null   object
 14  사고유형       4916 non-null   object
 15  법규위반사항     4916 non-null   object
 16  기상상태       4916 non-null   object
 17  사망자수       4916 non-null   int64 
 18  부상자수       4916 non-null   int64 
dtypes: int64(2), object(17)
memory usage: 729.8+ KB


In [ ]:
rData.describe()

,사망자수,부상자수
count,4916.000000,4916.000000
mean,0.068755,5.747356
std,0.259415,1.952238
min,0.000000,2.000000
25%,0.000000,4.000000
50%,0.000000,5.000000
75%,0.000000,7.000000
max,2.000000,16.000000


In [ ]:
rData.describe(include = 'object')

,다발지구분,다발지시군구,지점명,발생일,발생시간대,사고내용,가해자성별,가해자연령,가해자차종,가해자신체상해정도,피해자성별,피해자연령,피해자차종,피해자신체상해정도,사고유형,법규위반사항,기상상태
count,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916,4916
unique,3,461,930,1056,24,4,3,90,13,6,4,96,13,7,13,7,5
top,2019년 다발지,서울특별시 영등포구1,경기도 성남시 중원구 성남동(모란사거리 부근),2020-10-29,17시,경상,남,불명,자전거,상해없음,남,59세,자전거,경상,차대차 - 측면충돌,안전운전 의무 불이행,맑음
freq,1727,30,22,16,380,2631,3602,148,2201,3007,3365,136,3035,1918,2491,3169,4635


In [ ]:
df = pd.DataFrame()

In [ ]:
df['Year'] = rData['발생일'].str.split("-").str[0]
df['Month'] = rData['발생일'].str.split("-").str[1]
df['Date'] = rData['발생일'].str.split("-").str[2]
df['Time'] = rData['발생시간대'].str.replace("시", "")
df['Place'] = rData['지점명']
df['accedentType'] = rData['사고유형']
df['accedentDetail'] = rData['사고내용']



In [ ]:
tempPrepAge = rData['가해자연령'].copy()
tempPrepAge = tempPrepAge.replace("불명", None)
tempPrepAge = tempPrepAge.str.replace("세", "")


df['prepAge'] = pd.to_numeric(tempPrepAge)
df['prepGender'] = rData['가해자성별']
df['prepVehicle'] = rData['가해자차종']
df['prepDamege'] = rData['가해자신체상해정도']



In [ ]:
tempVictemAge = rData['피해자연령'].copy()
tempVictemAge = tempVictemAge.replace("불명", None)
tempVictemAge = tempVictemAge.replace("없음", None)
tempVictemAge = tempVictemAge.str.replace("세", "")

df['victemAge'] = pd.to_numeric(tempVictemAge)
df['victemGender'] = rData['피해자성별']
df['victemVehicle'] = rData['피해자차종']
df['victemDamege'] = rData['피해자신체상해정도']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            4916 non-null   object 
 1   Month           4916 non-null   object 
 2   Date            4916 non-null   object 
 3   Time            4916 non-null   object 
 4   Place           4916 non-null   object 
 5   accedentType    4916 non-null   object 
 6   accedentDetail  4916 non-null   object 
 7   prepAge         4768 non-null   float64
 8   prepGender      4916 non-null   object 
 9   prepVehicle     4916 non-null   object 
 10  prepDamege      4916 non-null   object 
 11  victemAge       4838 non-null   float64
 12  victemGender    4916 non-null   object 
 13  victemVehicle   4916 non-null   object 
 14  victemDamege    4916 non-null   object 
dtypes: float64(2), object(13)
memory usage: 576.2+ KB


In [ ]:
rData['가해자성별'].describe()

,가해자성별
count,4916
unique,3
top,남
freq,3602


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    4916 non-null   object
 1   Month   4916 non-null   object
 2   Date    4916 non-null   object
dtypes: object(3)
memory usage: 115.3+ KB


In [ ]:
rData.Year

,Year
0,2018
1,2018
2,2018
3,2018
4,2018
...,...
4911,2020
4912,2020
4913,2020
4914,2020


In [ ]:
rData.Month

,Month
0,03
1,03
2,06
3,11
4,04
...,...
4911,12
4912,03
4913,03
4914,06


In [ ]:
rData.Date

,Date
0,23
1,25
2,23
3,09
4,03
...,...
4911,31
4912,14
4913,31
4914,24


In [ ]:
rData.isna().sum()

,0
다발지구분,0
다발지시군구,0
지점명,0
발생일,0
발생시간대,0
사고내용,0
가해자성별,0
가해자연령,0
가해자차종,0
가해자신체상해정도,0


In [ ]:
rData['피해자연령'] = rData['피해자연령'].str.replace('세', '').astype(int)

ValueError: invalid literal for int() with base 10: '불명'

In [ ]:
# 불명 아닌 데이터만 필터링
filtered = rData[rData['가해자연령'] != '불명']

# 문자열에서 '세' 제거 → 정수형으로 변환
filtered['가해자연령'] = filtered['가해자연령'].str.replace('세', '', regex=False).astype(int)

print(filtered)


          다발지구분      다발지시군구                           지점명         발생일 발생시간대  \
0     2018년 다발지  서울특별시 종로구1    서울특별시 종로구 종로6가(흥인지문사거리 부근)  2018-03-23   07시   
1     2018년 다발지  서울특별시 종로구1    서울특별시 종로구 종로6가(흥인지문사거리 부근)  2018-03-25   17시   
2     2018년 다발지  서울특별시 종로구1    서울특별시 종로구 종로6가(흥인지문사거리 부근)  2018-06-23   14시   
3     2018년 다발지  서울특별시 종로구1    서울특별시 종로구 종로6가(흥인지문사거리 부근)  2018-11-09   18시   
4     2018년 다발지  서울특별시 용산구1  서울특별시 용산구 서빙고동(반포대교북단교차로 부근)  2018-04-03   15시   
...         ...         ...                           ...         ...   ...   
4911  2020년 다발지   경상남도 거제시2       경상남도 거제시 장평동(장평동493 부근)  2020-12-31   18시   
4912  2020년 다발지   경상남도 거창군1         경상남도 거창군 거창읍(거창2교 부근)  2020-03-14   10시   
4913  2020년 다발지   경상남도 거창군1         경상남도 거창군 거창읍(거창2교 부근)  2020-03-31   08시   
4914  2020년 다발지   경상남도 거창군1         경상남도 거창군 거창읍(거창2교 부근)  2020-06-24   15시   
4915  2020년 다발지   경상남도 거창군1         경상남도 거창군 거창읍(거창2교 부근)  2020-09-27   12시   

      사고내용 가해자성별  가해자연령 가해자차종 가해자신체상해정도  ... 피해자차종 

<ipython-input-62-58104d57279d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['가해자연령'] = filtered['가해자연령'].str.replace('세', '', regex=False).astype(int)
